# Question Answering

## Overview

주어진 질문과 관련된 문서를 검색하는 방법에 대해 살펴봤으니,
다음으로는 해당 문서를 가져와 원래 질과 함께서 언어 모델에 전달하고 질문에 답하도록 요청하는 입니다..

![overview.jpeg](overview.jpeg)

<img src="fig15.png" width="500">

* 벡터 스토어에서 여러 관련 문서가 검색되었습니다. 
* 관련 문서들을 압축하여 LLM 컨텍스트에 맞춥니다. 
* 질문과 함께 정보를 LLM으로 보내고 받은 응답을 포맷합니다.니다.

### RetrievalQA chain

<img src="fig16.png" width="400">

이에 대한 일반적인 흐름은 다음과 같습니다. 
질문이 들어오면 관련 문서를 찾은 다음 시스템 프롬프트와 질문과 함께 해당 분할을 언어 모델에 전달하고 답변을 습니다다.

기본적으로 모든 청크를 동일한 컨텍스트 창, 동일한 언어 모델로합니다달한다. 이러한 방법에는 장단점이 있는데, 단점은 문서가 너무 많아 동일한 컨텍스트 창에 문서를 모두 전달할 수 없는 경우가 있다입니다점이다.

In [276]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [277]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(
    persist_directory=persist_directory, 
    embedding_function=embedding
)

In [278]:
print(vectordb._collection.count())

208


In [279]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [280]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [281]:
from langchain.chains import RetrievalQA

In [282]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [283]:
result = qa_chain.invoke(question)

In [284]:
result["result"]

'The major topics for this class include machine learning, statistics, and algebra. In addition to these main topics, there will be discussions covering extensions of the material taught in the main lectures.'

### Prompt

In [286]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [287]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [288]:
question = "Is probability a class topic?"

In [289]:
result = qa_chain.invoke(question)

In [290]:
result["result"]

'Yes, probability is a class topic assumed to be familiar to students, as mentioned by the instructor. Thanks for asking!'

In [291]:
result["source_documents"][0]

Document(metadata={'page': 4, 'source': 'docs/cs229_lectures/MachineLearning-Lecture01.pdf'}, page_content="of this class will not be very programming intensive, although we will do some \nprogramming, mostly in either MATLAB or Octave. I'll say a bit more about that later.  \nI also assume familiarity with basic probability and statistics. So most undergraduate \nstatistics class, like Stat 116 taught here at Stanford, will be more than enough. I'm gonna \nassume all of you know what random variables are, that all of you know what expectation \nis, what a variance or a random variable is. And in case of some of you, it's been a while \nsince you've seen some of this material. At some of the discussion sections, we'll actually \ngo over some of the prerequisites, sort of as a refresher course under prerequisite class. \nI'll say a bit more about that later as well.  \nLastly, I also assume familiarity with basic linear algebra. And again, most undergraduate \nlinear algebra courses are

### RetrievalQA chain types - Map Reduce

지금까지 기본적으로 사용하는 기술인 stuff 기술을 사용했는데,
기본적으로 모든 문서를 최종 프롬프트에 채워넣는 방법이다.
이는 언어 모델을 한 번만 호출하면 되기 때문에 정말 좋은 방법이다.

그러나 문서가 너무 많으면 컨텍스트 창에 모두 들어가지 못할 수 있다는 제한이 문서에 대한 질문 답변을 수행하는 데 사용할 수 있는 다른 유형의 기술에는 Mat_reduce, Refine, Map_rerank 등이 있으며, 이들은 언어 모델에 대한 더 많은 호출이 포함되지만 임의의 많은 문서에 대해 작동할 수 있다는 이점이 있습니다.

<img src="fig17.png" width="400">술이다.이다.

In [293]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)

In [294]:
result = qa_chain_mr.invoke(question)

In [295]:
result["result"]

'Yes, probability is a class topic in the context of machine learning algorithms. The instructor assumes familiarity with basic probability and statistics, and a probabilistic interpretation is used to derive the first classification algorithm, indicating that probability is covered in the class.'

If you wish to experiment on the `LangChain plus platform`:

 * Go to [langchain plus platform](https://smith.langchain.com/) and sign up
 * Create an API key from your account's settings
 * Use this API key in the code below   
 * uncomment the code  
 Note, the endpoint in the video differs from the one below. Use the one below.

In [297]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "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"
from uuid import uuid4
unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"

In [298]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)
result = qa_chain_mr.invoke(question)
result["result"]

'Yes, probability is a class topic based on the information provided in the document.'

In [299]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="refine"
)
result = qa_chain_mr.invoke(question)
result["result"]

'Given the additional context provided, it is clear that probability will indeed be a class topic in the context of machine learning. The instructor will use a probabilistic interpretation to introduce classification algorithms, building on concepts from statistics and possibly algebra. The discussion sections will serve as a refresher for these foundational topics and also provide extensions to the material covered in the main lectures, allowing for a deeper exploration of the concepts within the field of machine learning.'

### RetrievalQA limitations
 
QA fails to preserve conversational history.

In [301]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [302]:
question = "Is probability a class topic?"
result = qa_chain.invoke(question)
result["result"]

'Yes, probability is a class topic in the course being described. The instructor assumes familiarity with basic probability and statistics, so it is likely that probability concepts will be covered in the class.'

In [303]:
question = "why are those prerequesites needed?"
result = qa_chain.invoke(question)
result["result"]

'The prerequisites mentioned in the context are needed because the course assumes familiarity with basic concepts in computer science, probability and statistics, and linear algebra. These prerequisites are essential for understanding and applying the machine learning algorithms taught in the class effectively.'